In [36]:
# coding: utf-8

# spectral subtraction: one of noise reduction method
#                       This is a simple python script, is not advanced one.
#
# Usage:
#    specify wav file names below,
#       infile: input wav file including noise
#       outfile: output wav file
#       noisefile: noise only wav file, that is some (noise only) portion of input wav edited by manual (ex: Audacity)
# 
#     then, python3 ss1.py

# Check version:
# 
#  Python 3.6.4 win32 64bit
#  Windows 10, 64bit
#  numpy (1.14.0)
#  scipy (1.0.0)
#  librosa (0.6.0)

import numpy as np
import scipy.io.wavfile
import librosa
import sounddevice as sd
import soundfile as sf
import matplotlib.pyplot as plot

import warnings
warnings.simplefilter("ignore", DeprecationWarning)


# edit following wav file name
infile='noisy_speech.wav'

#data, fs = sf.read(infile, dtype='float32')  
#sd.play(data, fs)
#status = sd.wait()  # Wait until file is done playing

outfile='output_short.wav'


noisefile='noise_short.wav'


# load input file, and stft (Short-time Fourier transform)
print ('load wav', infile)
w, sr = librosa.load( infile, sr=None, mono=True) # keep native sr (sampling rate) and trans into mono
s= librosa.stft(w)    # Short-time Fourier transform
ss= np.abs(s)         # get magnitude
angle= np.angle(s)    # get phase
b=np.exp(1.0j* angle) # use this phase information when Inverse Transform

# load noise only file, stft, and get mean
print ('load wav', noisefile)
nw, nsr = librosa.load( noisefile, sr=None, mono=True)
ns= librosa.stft(nw) 
nss= np.abs(ns)
mns= np.mean(nss, axis=1) # get mean

# subtract noise spectral mean from input spectral, and istft (Inverse Short-Time Fourier Transform)
sa= ss - mns.reshape((mns.shape[0],1))  # reshape for broadcast to subtract
sa0= sa * b  # apply phase information
y= librosa.istft(sa0) # back to time domain signal

# save as a wav file
scipy.io.wavfile.write(outfile, sr, (y * 32768).astype(np.int16)) # save signed 16-bit WAV format
#librosa.output.write_wav(outfile, y , sr)  # save 32-bit floating-point WAV format, due to y is float 
sf.write(outfile, y, sr, 'PCM_24')
print ('write wav', outfile)
#data, fs = sf.read(outfile, dtype='float32')  
#sd.play(data, fs)
#status = sd.wait()  # Wait until file is done playing

 



load wav noisy_speech.wav
load wav noise_short.wav
write wav output_short.wav


In [40]:
import math
[f, fs] = librosa.load('clean.wav');
[f2, fs2] = librosa.load(outfile);
top = 1/len(f)*(sum(f**2))
bottom = 1/len(f2)*(sum(f2**2))
SDR = 10*math.log10(top/bottom)

In [41]:
SDR

-7.8347802038556305

In [42]:
import math
[f, fs] = librosa.load('clean.wav');
[f2, fs2] = librosa.load(noisefile);
top = 1/len(f)*(sum(f**2))
bottom = 1/len(f2)*(sum(f2**2))
SIR = 10*math.log10(top/bottom)

In [43]:
SIR

-11.042306742147757